In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nasket_data_by_sentences-2020-04-15 (1).xlsx


### Read data

In [2]:
df = pd.read_excel('/kaggle/input/nasket_data_by_sentences-2020-04-15 (1).xlsx')
df.head(10)

,id,sentence,Overall Sentiment\n(Positive/Negative/Neutral),"Services & Staff\n(0 = not mentioned, \n1 = mentioned)",Services & Staff Sentiment,"Amenities Sentiment(0 = not mentioned, \n1 = mentioned)",Amenities Sentiment,"Hotel Condition \n(0 = not mentioned, \n1 = mentioned)",Hotel Condition Sentiment,"Cleanliness\n(0 = not mentioned, \n1 = mentioned)",Cleanliness Sentiment
0,34437648,5 Stories with no elevator,Negative,1.0,Negative,0.0,Negative,1.0,Negative,0.0,Negative
1,34437648,Great location right at Nana BTS Station!,Positive,1.0,Positive,0.0,Positive,1.0,Positive,0.0,Positive
2,34437648,Room was modern and clean eventhough small.,Positive,1.0,Positive,0.0,Positive,1.0,Positive,1.0,Positive
3,34437648,Only gripe... stairs only and my room was on t...,Negative,0.0,NaN,0.0,NaN,1.0,Negative,0.0,Negative
4,34437886,Dump,Negative,0.0,NaN,0.0,NaN,1.0,Negative,0.0,Negative
5,34437886,pretty run down old hotel.,Negative,0.0,NaN,0.0,NaN,1.0,Negative,0.0,NaN
6,34437886,its nothing like what the photos depict.,Negative,0.0,NaN,0.0,Negative,1.0,Negative,0.0,NaN
7,34437886,book elsewhere.,Negative,0.0,NaN,0.0,Negative,1.0,Negative,0.0,NaN
8,36080259,"Amazing service, amazing staff, amazing room",Positive,1.0,Positive,0.0,Positive,1.0,Positive,0.0,NaN
9,36080259,This is hotel is conveniently located next to ...,Positive,0.0,Positive,0.0,Positive,0.0,NaN,0.0,NaN


### Filter data

In [3]:
reviews_df = df[['sentence','Overall Sentiment\n(Positive/Negative/Neutral)']]
reviews_df.head(10)

,sentence,Overall Sentiment\n(Positive/Negative/Neutral)
0,5 Stories with no elevator,Negative
1,Great location right at Nana BTS Station!,Positive
2,Room was modern and clean eventhough small.,Positive
3,Only gripe... stairs only and my room was on t...,Negative
4,Dump,Negative
5,pretty run down old hotel.,Negative
6,its nothing like what the photos depict.,Negative
7,book elsewhere.,Negative
8,"Amazing service, amazing staff, amazing room",Positive
9,This is hotel is conveniently located next to ...,Positive


## Pre-processing

In [4]:
import re
reviews_df['sentence'] = reviews_df['sentence'].str.replace('[^\w\s]','')
reviews_df['sentence'] = reviews_df['sentence'].str.replace('\d+', '')
reviews_df['sentence'] = reviews_df['sentence'].str.lower()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

### Drop empty cells

In [5]:
drop = reviews_df[pd.isnull(reviews_df['Overall Sentiment\n(Positive/Negative/Neutral)'])].index
reviews_df.drop(drop , inplace=True)
reviews_df = reviews_df.reset_index(drop = True) 
reviews_df.head(10)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,sentence,Overall Sentiment\n(Positive/Negative/Neutral)
0,stories with no elevator,Negative
1,great location right at nana bts station,Positive
2,room was modern and clean eventhough small,Positive
3,only gripe stairs only and my room was on the ...,Negative
4,dump,Negative
5,pretty run down old hotel,Negative
6,its nothing like what the photos depict,Negative
7,book elsewhere,Negative
8,amazing service amazing staff amazing room,Positive
9,this is hotel is conveniently located next to ...,Positive


In [6]:
reviews_df['sentence'].replace('', np.nan, inplace=True)
drop = reviews_df[pd.isnull(reviews_df['sentence'])].index
reviews_df.drop(drop , inplace=True)
reviews_df = reviews_df.reset_index(drop = True) 
reviews_df.head(10)

,sentence,Overall Sentiment\n(Positive/Negative/Neutral)
0,stories with no elevator,Negative
1,great location right at nana bts station,Positive
2,room was modern and clean eventhough small,Positive
3,only gripe stairs only and my room was on the ...,Negative
4,dump,Negative
5,pretty run down old hotel,Negative
6,its nothing like what the photos depict,Negative
7,book elsewhere,Negative
8,amazing service amazing staff amazing room,Positive
9,this is hotel is conveniently located next to ...,Positive


### Remove stop words

In [7]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [8]:
reviews_df['sentence'][60]

'very bright and clean room'

In [9]:
reviews_df['no_sw'] = reviews_df['sentence'][:].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [10]:
reviews_df.head(10)

,sentence,Overall Sentiment\n(Positive/Negative/Neutral),no_sw
0,stories with no elevator,Negative,stories elevator
1,great location right at nana bts station,Positive,great location right nana bts station
2,room was modern and clean eventhough small,Positive,room modern clean eventhough small
3,only gripe stairs only and my room was on the ...,Negative,gripe stairs room top floor th floor really ho...
4,dump,Negative,dump
5,pretty run down old hotel,Negative,pretty run old hotel
6,its nothing like what the photos depict,Negative,nothing like photos depict
7,book elsewhere,Negative,book elsewhere
8,amazing service amazing staff amazing room,Positive,amazing service amazing staff amazing room
9,this is hotel is conveniently located next to ...,Positive,hotel conveniently located next sky train exit...


### Spelling correction

In [11]:
!pip install pyspellchecker
from spellchecker import SpellChecker

spell = SpellChecker()

     |████████████████████████████████| 1.9 MB 4.8 MB/s 


## Bag of words

In [12]:
import nltk
from nltk import pos_tag, word_tokenize


sentence_vectors = []
wordfreq = {}
tokenized = [0]*len(reviews_df['no_sw'])
for sentence in reviews_df['no_sw']:
    tokens = word_tokenize(sentence)
    sent_vec = []
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1

In [13]:
import heapq
num_features = 2000
most_freq = heapq.nlargest(num_features, wordfreq, key=wordfreq.get)

In [14]:
sentence_vectors = []
for sentence in reviews_df['no_sw']:
    sentence_tokens = nltk.word_tokenize(sentence)
    sent_vec = []
    for token in most_freq:
        if token in sentence_tokens:
            sent_vec.append(1)
        else:
            sent_vec.append(0)
    sentence_vectors.append(sent_vec)

In [15]:
sentence_vectors = np.asarray(sentence_vectors)
sentence_vectors[0]

array([0, 0, 0, ..., 0, 0, 0])

## Building Model

In [16]:
from sklearn.model_selection import train_test_split
labels = reviews_df['Overall Sentiment\n(Positive/Negative/Neutral)']
X_train, X_test, y_train, y_test = train_test_split(sentence_vectors, labels, test_size=0.2, random_state=0)

### Sample with RFC

In [17]:
from sklearn.ensemble import RandomForestClassifier

text_classifier = RandomForestClassifier(n_estimators=200, random_state=0)
text_classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=200, random_state=0)

In [18]:
predictions = text_classifier.predict(X_test)

### Evaluations

In [19]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[227   0  70   0 141   1]
 [  2   0   0   0   1   0]
 [ 69   0  53   0  54   0]
 [  1   0   0   0   0   0]
 [100   2  33   0 325   0]
 [  4   0   1   0   2   0]]
              precision    recall  f1-score   support

    Negative       0.56      0.52      0.54       439
   Negative        0.00      0.00      0.00         3
     Neutral       0.34      0.30      0.32       176
    Neutral        0.00      0.00      0.00         1
    Positive       0.62      0.71      0.66       460
   Positive        0.00      0.00      0.00         7

    accuracy                           0.56      1086
   macro avg       0.25      0.25      0.25      1086
weighted avg       0.55      0.56      0.55      1086

0.5570902394106814


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
